
Dieses Notebook vektorisiert Reddit Posts mit einem verfügbaren Embedding Model über die Ollama API.
Rate Limit: 12 Requests pro Minute - ABER mit Batch-Processing (mehrere Texte pro Request)

In [7]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import pickle
import os
from datetime import datetime
import requests
import json
import time

In [8]:
# Ollama API Client mit Batch Processing für bessere Effizienz
class OllamaEmbeddings:
    HOST = "https://f2ki-h100-1.f2.htw-berlin.de"
    PORT = 11435
    TIMEOUT = 120
    # Rate Limit: 12 Requests pro Minute = 1 Request alle 5 Sekunden
    REQUEST_DELAY = 5.0
    
    @classmethod
    def get_embeddings_batch(cls, texts, model="mxbai-embed-large:latest"):
        """Generiert Embeddings für mehrere Texte in einem Request (BATCH)"""
        url = f"{cls.HOST}:{cls.PORT}/api/embed"
        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }
        
        # Stelle sicher, dass texts eine Liste von Strings ist
        if isinstance(texts, str):
            texts = [texts]
        
        # Bereinige Texte
        clean_texts = [str(text).strip() for text in texts if text and str(text).strip()]
        
        if not clean_texts:
            print("ERROR: Keine gültigen Texte zum Verarbeiten")
            return None
        
        # API Parameter für Batch-Processing
        payload = {
            "model": model,
            "input": clean_texts,  # Liste von Texten für Batch-Processing
            "truncate": True,
            "keep_alive": "5m"
        }
        
        try:
            response = requests.post(url, headers=headers, json=payload, timeout=cls.TIMEOUT)
            
            if response.status_code != 200:
                print(f"ERROR: Request failed with status {response.status_code}: {response.text}")
                return None
                
            result = response.json()
            
            # API gibt "embeddings" Array zurück mit einem Embedding pro Input-Text
            embeddings = result.get('embeddings', [])
            
            if embeddings and len(embeddings) == len(clean_texts):
                return [np.array(emb) for emb in embeddings]  # Liste von Embeddings
            else:
                print(f"ERROR: Erwartete {len(clean_texts)} Embeddings, erhielt {len(embeddings)}")
                return None
            
        except requests.exceptions.Timeout:
            print(f"ERROR: Timeout nach {cls.TIMEOUT} Sekunden")
            return None
        except requests.exceptions.ConnectionError:
            print("ERROR: Verbindungsfehler zum Server")
            return None
        except Exception as e:
            print(f"ERROR: Unerwarteter Fehler: {e}")
            return None
    
    @classmethod
    def get_embeddings(cls, text, model="mxbai-embed-large:latest"):
        """Generiert Embedding für einzelnen Text (Wrapper für Kompatibilität)"""
        result = cls.get_embeddings_batch([text], model)
        return result[0] if result else None
    
    @classmethod
    def wait_for_rate_limit(cls):
        """Wartet die erforderliche Zeit für Rate Limiting"""
        time.sleep(cls.REQUEST_DELAY)
    
    @classmethod
    def test_connection(cls):
        """Testet die Verbindung zum Server"""
        try:
            test_url = f"{cls.HOST}:{cls.PORT}/api/tags"
            response = requests.get(test_url, timeout=10)
            return response.status_code == 200
        except:
            return False

In [9]:
# Embedding Model Management - Direkte Auswahl verfügbarer Models
print("=== Embedding Model Management ===")

# Bekannte verfügbare Embedding Models (basierend auf API Response)
AVAILABLE_MODELS = {
    "mxbai-embed-large:latest": {
        "parameter_size": "334M",
        "size_mb": 638.5,
        "family": "bert",
        "description": "Sehr gut, großes Model"
    },
    "all-minilm:latest": {
        "parameter_size": "23M", 
        "size_mb": 43.8,
        "family": "bert",
        "description": "Schnell, kleines Model"
    }
}

# Wähle bevorzugtes Model (mxbai-embed-large für beste Qualität)
EMBEDDING_MODEL = "mxbai-embed-large:latest"
MODEL_FOUND = True

print(f"✓ Verwende bekanntes Embedding Model: {EMBEDDING_MODEL}")
model_info = AVAILABLE_MODELS[EMBEDDING_MODEL]
print(f"  - Parameter: {model_info['parameter_size']}")
print(f"  - Größe: {model_info['size_mb']:.1f} MB")
print(f"  - Familie: {model_info['family']}")
print(f"  - Beschreibung: {model_info['description']}")

# Test des Models
print(f"\n=== Model Test: {EMBEDDING_MODEL} ===")
try:
    # Disable SSL warnings
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
    print("Teste Model mit kurzem Text...")
    test_embedding = OllamaEmbeddings.get_embeddings("Test", model=EMBEDDING_MODEL)
    if test_embedding is not None and len(test_embedding) > 0:
        print(f"✓ Model funktioniert perfekt!")
        print(f"✓ Embedding-Dimension: {len(test_embedding)}")
    else:
        print("❌ Model antwortet nicht korrekt")
        print("Versuche alternatives Model...")
        EMBEDDING_MODEL = "all-minilm:latest"
        test_embedding = OllamaEmbeddings.get_embeddings("Test", model=EMBEDDING_MODEL)
        if test_embedding is not None and len(test_embedding) > 0:
            print(f"✓ Alternatives Model funktioniert: {EMBEDDING_MODEL}")
            print(f"✓ Embedding-Dimension: {len(test_embedding)}")
        else:
            print("❌ Auch alternatives Model funktioniert nicht")
            MODEL_FOUND = False
except Exception as e:
    print(f"❌ Model-Test fehlgeschlagen: {e}")
    MODEL_FOUND = False

# Finaler Status
if MODEL_FOUND:
    print(f"\n🎉 BEREIT: Verwende Embedding Model '{EMBEDDING_MODEL}'")
    print("Alle Voraussetzungen erfüllt - kann mit Vektorisierung beginnen!")
else:
    print(f"\n❌ FEHLER: Kein funktionierendes Embedding Model verfügbar!")

=== Embedding Model Management ===
✓ Verwende bekanntes Embedding Model: mxbai-embed-large:latest
  - Parameter: 334M
  - Größe: 638.5 MB
  - Familie: bert
  - Beschreibung: Sehr gut, großes Model

=== Model Test: mxbai-embed-large:latest ===
Teste Model mit kurzem Text...
✓ Model funktioniert perfekt!
✓ Embedding-Dimension: 1024

🎉 BEREIT: Verwende Embedding Model 'mxbai-embed-large:latest'
Alle Voraussetzungen erfüllt - kann mit Vektorisierung beginnen!


In [10]:
# Daten laden nur wenn Model verfügbar
if MODEL_FOUND:
    print("Lade CSV Datei...")
    try:
        df = pd.read_csv("data/tesla_preprocessed.csv")
        print(f"✓ Datensatz geladen: {len(df)} Zeilen")
    except Exception as e:
        print(f"Fehler beim Laden der CSV: {e}")
        MODEL_FOUND = False
else:
    print("⚠ Überspringe Datenladung - kein funktionierendes Model verfügbar")

Lade CSV Datei...
✓ Datensatz geladen: 13268 Zeilen


In [11]:
# Text vorbereiten mit Batch-Kalkulation
if MODEL_FOUND and 'df' in locals():
    print("Bereite Texte vor...")
    df['combined_text'] = df['title'].fillna('') + ' ' + df['text'].fillna('')
    df['combined_text'] = df['combined_text'].str.strip()
    df = df[df['combined_text'] != '']

    print(f"Nach Bereinigung: {len(df)} Texte")

    # Batch-Größe für effizientere Verarbeitung
    BATCH_SIZE = 10  # 10 Texte pro Request
    num_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE  # Aufrunden
    
    # Schätze die benötigte Zeit mit Batch-Processing
    estimated_minutes = (num_batches * OllamaEmbeddings.REQUEST_DELAY) / 60
    estimated_hours = estimated_minutes / 60
    texts_per_minute = (BATCH_SIZE * 60) / OllamaEmbeddings.REQUEST_DELAY
    
    print(f"📊 Batch-Processing Konfiguration:")
    print(f"  - Batch-Größe: {BATCH_SIZE} Texte pro Request")
    print(f"  - Anzahl Batches: {num_batches}")
    print(f"  - Texte pro Minute: {texts_per_minute:.0f}")
    print(f"  - Geschätzte Zeit: {estimated_minutes:.1f} Minuten ({estimated_hours:.1f} Stunden)")
    
    # Warnung bei sehr langen Zeiten
    if estimated_hours > 2:
        print("⚠ WARNUNG: Lange Verarbeitungszeit!")
        print("Optionen:")
        print("  - Kleinere Stichprobe: df.head(1000)")
        print("  - Größere Batches (bis zu ~50 je nach Textlänge)")
        
        # Automatische Reduktion für Demo
        if len(df) > 2000:
            print(f"Verwende automatisch erste 1000 Texte für Demo (statt {len(df)})")
            df = df.head(1000)
            num_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE
            estimated_minutes = (num_batches * OllamaEmbeddings.REQUEST_DELAY) / 60
            print(f"Neue geschätzte Zeit: {estimated_minutes:.1f} Minuten")
else:
    print("⚠ Überspringe Textvorbereitung - Voraussetzungen nicht erfüllt")

Bereite Texte vor...
Nach Bereinigung: 13268 Texte
📊 Batch-Processing Konfiguration:
  - Batch-Größe: 10 Texte pro Request
  - Anzahl Batches: 1327
  - Texte pro Minute: 120
  - Geschätzte Zeit: 110.6 Minuten (1.8 Stunden)


In [12]:
# Batch-Vektorisierung für deutlich bessere Effizienz
if MODEL_FOUND and 'df' in locals():
    texts = df['combined_text'].tolist()
    
    print(f"🚀 Starte Batch-Vektorisierung von {len(texts)} Texten...")
    print(f"Model: {EMBEDDING_MODEL}")
    print(f"Batch-Größe: {BATCH_SIZE} Texte pro Request")
    print(f"Rate Limit: 12 Requests/Minute = {texts_per_minute:.0f} Texte/Minute")

    start_time = datetime.now()
    all_embeddings = []
    failed_indices = []
    
    # Verarbeite Texte in Batches
    for batch_idx in range(0, len(texts), BATCH_SIZE):
        batch_end = min(batch_idx + BATCH_SIZE, len(texts))
        batch_texts = texts[batch_idx:batch_end]
        batch_indices = list(range(batch_idx, batch_end))
        
        # Rate Limiting: Warte zwischen Requests (außer beim ersten)
        if batch_idx > 0:
            OllamaEmbeddings.wait_for_rate_limit()
        
        # Batch-Embedding generieren
        batch_embeddings = OllamaEmbeddings.get_embeddings_batch(batch_texts, model=EMBEDDING_MODEL)
        
        if batch_embeddings and len(batch_embeddings) == len(batch_texts):
            all_embeddings.extend(batch_embeddings)
        else:
            print(f"Fehler bei Batch {batch_idx//BATCH_SIZE + 1}, verwende Null-Vektoren...")
            failed_indices.extend(batch_indices)
            
            # Fallback: Null-Vektoren für alle Texte im fehlgeschlagenen Batch
            for _ in batch_texts:
                if all_embeddings:
                    null_embedding = np.zeros_like(all_embeddings[0])
                else:
                    null_embedding = np.zeros(1024)  # Standard-Dimension für mxbai-embed-large
                all_embeddings.append(null_embedding)
        
        # Progress-Update
        current_batch = (batch_idx // BATCH_SIZE) + 1
        total_batches = (len(texts) + BATCH_SIZE - 1) // BATCH_SIZE
        texts_processed = min(batch_end, len(texts))
        
        if current_batch % 5 == 0 or current_batch == total_batches:  # Alle 5 Batches oder am Ende
            elapsed = datetime.now() - start_time
            rate = texts_processed / elapsed.total_seconds() * 60  # Texte pro Minute
            remaining_texts = len(texts) - texts_processed
            eta_seconds = (remaining_texts / texts_per_minute) * 60
            eta_minutes = eta_seconds / 60
            
            print(f"Fortschritt: Batch {current_batch}/{total_batches} | "
                  f"Texte: {texts_processed}/{len(texts)} | "
                  f"Rate: {rate:.0f}/min | ETA: {eta_minutes:.1f}min")
        
        # Stopp bei zu vielen aufeinanderfolgenden Fehlern
        if len(failed_indices) > 50 and texts_processed < 200:
            print(f"\n⚠ STOPP: Zu viele frühe Fehler ({len(failed_indices)}) - prüfe Model und API")
            break

    print(f"\n✅ Batch-Vektorisierung abgeschlossen!")
    print(f"Verarbeitete Texte: {len(all_embeddings)}")
    
    if failed_indices:
        print(f"⚠ Warnung: {len(failed_indices)} Texte konnten nicht vektorisiert werden")
        if len(failed_indices) <= 20:
            print(f"Fehlerhafte Indizes: {failed_indices}")
        else:
            print(f"Erste 20 fehlerhafte Indizes: {failed_indices[:20]}...")
else:
    print("⚠ Überspringe Vektorisierung - Voraussetzungen nicht erfüllt")

🚀 Starte Batch-Vektorisierung von 13268 Texten...
Model: mxbai-embed-large:latest
Batch-Größe: 10 Texte pro Request
Rate Limit: 12 Requests/Minute = 120 Texte/Minute
Fortschritt: Batch 5/1327 | Texte: 50/13268 | Rate: 65/min | ETA: 110.2min
Fortschritt: Batch 10/1327 | Texte: 100/13268 | Rate: 62/min | ETA: 109.7min
Fortschritt: Batch 15/1327 | Texte: 150/13268 | Rate: 62/min | ETA: 109.3min
Fortschritt: Batch 20/1327 | Texte: 200/13268 | Rate: 61/min | ETA: 108.9min
Fortschritt: Batch 25/1327 | Texte: 250/13268 | Rate: 61/min | ETA: 108.5min
Fortschritt: Batch 30/1327 | Texte: 300/13268 | Rate: 61/min | ETA: 108.1min
Fortschritt: Batch 35/1327 | Texte: 350/13268 | Rate: 61/min | ETA: 107.7min
Fortschritt: Batch 40/1327 | Texte: 400/13268 | Rate: 61/min | ETA: 107.2min
Fortschritt: Batch 45/1327 | Texte: 450/13268 | Rate: 61/min | ETA: 106.8min
Fortschritt: Batch 50/1327 | Texte: 500/13268 | Rate: 60/min | ETA: 106.4min
Fortschritt: Batch 55/1327 | Texte: 550/13268 | Rate: 60/min | ETA

In [13]:
# Embeddings zusammenführen und speichern
if 'all_embeddings' in locals() and all_embeddings:
    embeddings_array = np.vstack(all_embeddings)
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n✓ Vektorisierung abgeschlossen!")
    print(f"Dauer: {duration}")
    print(f"Embeddings Shape: {embeddings_array.shape}")
    print(f"Erfolgreiche Vektorisierungen: {len(all_embeddings) - len(failed_indices)}/{len(texts)}")
    
    # Speichern mit Model-spezifischen Namen
    model_suffix = EMBEDDING_MODEL.replace(':', '_').replace('/', '_').replace('-', '_')
    print("Speichere Embeddings...")
    np.save(f'data/embedding_data/Reddit_embeddings_{model_suffix}.npy', embeddings_array)
    
    metadata = {
        'model_name': EMBEDDING_MODEL,
        'api_host': f"{OllamaEmbeddings.HOST}:{OllamaEmbeddings.PORT}",
        'embedding_dimension': embeddings_array.shape[1],
        'num_texts': embeddings_array.shape[0],
        'processing_time': str(duration),
        'failed_indices': failed_indices,
        'success_rate': (len(all_embeddings) - len(failed_indices)) / len(texts) if 'texts' in locals() else 0,
        'rate_limit': '12 requests/minute',
        'request_delay': OllamaEmbeddings.REQUEST_DELAY
    }
    
    with open(f'data/embedding_data/embedding_metadata_{model_suffix}.pkl', 'wb') as f:
        pickle.dump(metadata, f)
    
    if 'df' in locals():
        df_reduced = df[['title','text','score','created']].reset_index(drop=True)
        df_reduced.to_csv(f'data/embedding_data/Reddit_metadata_{model_suffix}.csv', index=False)
    
    print(f"✓ Embeddings gespeichert als 'data/embedding_data/Reddit_embeddings_{model_suffix}.npy'")
    print(f"✓ Metadata gespeichert als 'data/embedding_data/embedding_metadata_{model_suffix}.pkl'")
    print(f"✓ Daten gespeichert als 'data/embedding_data/Reddit_metadata_{model_suffix}.csv'")
    print(f"Embedding-Dimension: {embeddings_array.shape[1]}")
    
else:
    print("FEHLER: Keine Embeddings generiert!")


✓ Vektorisierung abgeschlossen!
Dauer: 3:17:41.849693
Embeddings Shape: (13268, 1024)
Erfolgreiche Vektorisierungen: 10390/13268
Speichere Embeddings...
✓ Embeddings gespeichert als 'data/embedding_data/Reddit_embeddings_mxbai_embed_large_latest.npy'
✓ Metadata gespeichert als 'data/embedding_data/embedding_metadata_mxbai_embed_large_latest.pkl'
✓ Daten gespeichert als 'data/embedding_data/Reddit_metadata_mxbai_embed_large_latest.csv'
Embedding-Dimension: 1024


In [14]:
# Optionale Validierung der generierten Embeddings
if 'embeddings_array' in locals():
    print("\n=== Embedding Validierung ===")
    print(f"Shape: {embeddings_array.shape}")
    print(f"Datentyp: {embeddings_array.dtype}")
    print(f"Min/Max Werte: {embeddings_array.min():.4f} / {embeddings_array.max():.4f}")
    print(f"Durchschnittliche Norm: {np.linalg.norm(embeddings_array, axis=1).mean():.4f}")
    
    # Prüfe auf Null-Vektoren
    null_vectors = np.sum(np.all(embeddings_array == 0, axis=1))
    print(f"Null-Vektoren: {null_vectors}/{len(embeddings_array)}")
    
    if null_vectors == len(failed_indices):
        print("✓ Anzahl Null-Vektoren entspricht den fehlgeschlagenen Requests")
    else:
        print("⚠ Unerwartete Anzahl an Null-Vektoren")


=== Embedding Validierung ===
Shape: (13268, 1024)
Datentyp: float64
Min/Max Werte: -0.1393 / 0.2989
Durchschnittliche Norm: 0.7831
Null-Vektoren: 2878/13268
✓ Anzahl Null-Vektoren entspricht den fehlgeschlagenen Requests


In [15]:
# Teste verschiedene Batch-Größen um das Optimum zu finden
if MODEL_FOUND and 'df' in locals():
    print("=== Batch-Größen Test ===")
    
    # Prüfe ob die Batch-Methode verfügbar ist
    if not hasattr(OllamaEmbeddings, 'get_embeddings_batch'):
        print("❌ FEHLER: get_embeddings_batch Methode nicht gefunden!")
        print("🔧 LÖSUNG: Bitte führe Cell 3 (OllamaEmbeddings Klasse) erneut aus")
        print("   oder starte den Kernel neu: Kernel → Restart Kernel")
    else:
        # Test-Texte (erste 100 für schnellen Test)
        test_texts = df['combined_text'].head(100).tolist()
        
        # Verschiedene Batch-Größen testen
        test_batch_sizes = [1, 5, 10, 20, 30, 50]
        successful_batches = []
        
        for batch_size in test_batch_sizes:
            print(f"\nTeste Batch-Größe: {batch_size}")
            
            # Nimm erste X Texte für Test
            test_batch = test_texts[:batch_size]
            
            start_time = datetime.now()
            try:
                result = OllamaEmbeddings.get_embeddings_batch(test_batch, model=EMBEDDING_MODEL)
                end_time = datetime.now()
                duration = (end_time - start_time).total_seconds()
                
                if result:
                    print(f"  ✓ Erfolgreich: {len(result)} Embeddings in {duration:.2f}s")
                    print(f"  ✓ Rate: {len(result)/duration:.1f} Texte/Sekunde")
                    print(f"  ✓ Embedding Shape: {result[0].shape}")
                    successful_batches.append((batch_size, len(result)/duration))
                else:
                    print(f"  ❌ Fehlgeschlagen")
                    break
                    
            except Exception as e:
                print(f"  ❌ Fehler: {e}")
                break
        
        # Empfehlungen basierend auf erfolgreichen Tests
        if successful_batches:
            print(f"\n💡 Test-Ergebnisse für {EMBEDDING_MODEL}:")
            best_batch = max(successful_batches, key=lambda x: x[1])
            print(f"✅ Schnellste Batch-Größe: {best_batch[0]} ({best_batch[1]:.1f} Texte/s)")
            
            # Empfehlungen
            print(f"\n📊 Empfohlene Batch-Größen:")
            if best_batch[0] <= 5:
                print("• Konservativ: 5-10 (sichere Wahl)")
                print("• Empfohlen: 10-15 (ausgewogen)")
            elif best_batch[0] <= 20:
                print("• Konservativ: 10-15 (sichere Wahl)")
                print("• Empfohlen: 15-25 (ausgewogen)")
                print("• Aggressiv: 25-40 (maximale Geschwindigkeit)")
            else:
                print("• Empfohlen: 20-30 (ausgewogen)")
                print("• Aggressiv: 30-50 (maximale Geschwindigkeit)")
            
            # Aktualisiere BATCH_SIZE für optimale Performance
            optimal_batch = min(best_batch[0] * 2, 50)  # Doppelt, aber max 50
            print(f"\n🚀 Optimierte Batch-Größe für Produktion: {optimal_batch}")
            
            # Globale Variable für andere Cells setzen
            globals()['OPTIMAL_BATCH_SIZE'] = optimal_batch
        else:
            print("\n❌ Keine erfolgreichen Batch-Tests")
else:
    print("⚠ Batch-Test übersprungen - Voraussetzungen nicht erfüllt")

=== Batch-Größen Test ===

Teste Batch-Größe: 1
ERROR: Verbindungsfehler zum Server
  ❌ Fehlgeschlagen

❌ Keine erfolgreichen Batch-Tests
